In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import statsmodels.api as sm
import statsmodels.formula.api as smf

/Users/apple/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
vol = pd.read_csv('oxfordmanrealizedvolatilityindices.csv',parse_dates=[0])

In [3]:
spy = pd.read_csv('SPY.csv', index_col = [0])

In [4]:
irx = pd.read_csv('^IRX.csv', index_col = [0])

In [5]:
spy.dropna(inplace=True)

In [6]:
vol=vol.loc[vol.Symbol=='.SPX']
vol.head()

,Unnamed: 0,Symbol,rk_twoscale,rk_th2,rv5,nobs,medrv,rsv,bv_ss,rv10_ss,bv,close_price,rk_parzen,rsv_ss,rv5_ss,open_price,open_time,open_to_close,rv10,close_time
108878,2000-01-03,.SPX,0.000122,0.000130,0.000141,1554.0,0.000058,0.000096,0.000141,0.000164,0.000141,1454.24,0.000340,0.000096,0.000141,1471.21,83004.0,-0.011602,0.000164,145959.0
108879,2000-01-04,.SPX,0.000142,0.000162,0.000224,1564.0,0.000067,0.000180,0.000173,0.000295,0.000173,1399.02,0.000215,0.000180,0.000224,1449.00,83009.0,-0.035102,0.000295,150000.0
108880,2000-01-05,.SPX,0.000219,0.000240,0.000314,1552.0,0.000142,0.000131,0.000303,0.000267,0.000303,1402.09,0.000163,0.000131,0.000314,1395.62,83012.0,0.004625,0.000267,150003.0
108881,2000-01-06,.SPX,0.000127,0.000132,0.000131,1561.0,0.000070,0.000056,0.000128,0.000115,0.000128,1403.06,0.000067,0.000056,0.000131,1398.87,83005.0,0.002991,0.000115,145954.0
108882,2000-01-07,.SPX,0.000091,0.000095,0.000094,1540.0,0.000049,0.000023,0.000079,0.000121,0.000079,1440.84,0.000215,0.000023,0.000094,1403.45,83000.0,0.026293,0.000121,150000.0


In [7]:
vol.rename(index=str, columns={"Unnamed: 0": "Date"}, inplace=True)
vol.loc[vol['Date'].dt.hour==23, 'Date'] = vol['Date'] + dt.timedelta(hours=1)
vol = vol[['Date','rv5']]

In [8]:
vol.set_index('Date', inplace=True)
vol.head()

,rv5
Date,
2000-01-03,0.000141
2000-01-04,0.000224
2000-01-05,0.000314
2000-01-06,0.000131
2000-01-07,0.000094


In [9]:
spy['ret'] = spy['Adj Close'].pct_change()
irx['ret'] = irx['Adj Close']/1200
irx = irx.loc['2001-01-02':]
spy = spy.loc['2001-01-02':]

In [10]:
vol = vol.loc['2000-12-29':]
vol.head()

,rv5
Date,
2000-12-29,0.000091
2001-01-02,0.000203
2001-01-03,0.001003
2001-01-04,0.000159
2001-01-05,0.000244


In [11]:
spy.head()

,Open,High,Low,Close,Adj Close,Volume,ret
Date,,,,,,,
2001-02-01,137.100006,137.990005,121.800003,123.949997,88.646049,178607000,-0.095388
2001-03-01,124.050003,127.750000,108.040001,116.690002,83.453873,318187200,-0.058572
2001-04-01,116.300003,127.269997,109.300003,126.660004,90.828072,251839700,0.088363
2001-05-01,125.070000,132.089996,123.440002,125.949997,90.318939,208040000,-0.005605
2001-06-01,126.199997,129.229996,120.400002,122.599998,87.916611,203578200,-0.026598


In [12]:
irx.head()

,Open,High,Low,Close,Adj Close,Volume,ret
Date,,,,,,,
2001-02-01,4.83,4.95,4.62,4.72,4.72,0,0.003933
2001-03-01,4.70,4.75,4.13,4.18,4.18,0,0.003483
2001-04-01,4.17,4.19,3.59,3.83,3.83,0,0.003192
2001-05-01,3.77,3.82,3.47,3.54,3.54,0,0.002950
2001-06-01,3.53,3.60,3.33,3.56,3.56,0,0.002967


In [13]:
data = pd.DataFrame(index = spy.index)
data['ret'] = (spy['ret'] - irx['ret'])
data['ret_sqr'] = data['ret'] ** 2
data.dropna(inplace=True)
data.head()

,ret,ret_sqr
Date,,
2001-02-01,-0.099321,0.009865
2001-03-01,-0.062055,0.003851
2001-04-01,0.085171,0.007254
2001-05-01,-0.008555,0.000073
2001-06-01,-0.029565,0.000874


In [14]:
vol['var_lag'] = vol['rv5'].shift()
vol.dropna(inplace=True)
vol = vol.loc[:'2018-05-09']
vol.head()

,rv5,var_lag
Date,,
2001-01-02,0.000203,0.000091
2001-01-03,0.001003,0.000203
2001-01-04,0.000159,0.001003
2001-01-05,0.000244,0.000159
2001-01-08,0.000083,0.000244


In [15]:
vol.shape

(4356, 2)

In [16]:
data.shape

(210, 2)

In [17]:
data['date'] = data.index
data.head()

,ret,ret_sqr,date
Date,,,
2001-02-01,-0.099321,0.009865,2001-02-01
2001-03-01,-0.062055,0.003851,2001-03-01
2001-04-01,0.085171,0.007254,2001-04-01
2001-05-01,-0.008555,0.000073,2001-05-01
2001-06-01,-0.029565,0.000874,2001-06-01


In [18]:
vol['date'] = vol.index
vol['date'] = vol['date'].dt.date

In [19]:
vol['date'] = vol['date'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [20]:
data = data.merge(vol, how = 'left', left_on = 'date', right_on = 'date')
data.dropna(inplace=True)

In [21]:
data.set_index('date')

,ret,ret_sqr,rv5,var_lag
date,,,,
2001-02-01,-0.099321,9.864641e-03,0.000073,0.000112
2001-03-01,-0.062055,3.850864e-03,0.000270,0.000216
2001-05-01,-0.008555,7.319587e-05,0.000095,0.000079
2001-06-01,-0.029565,8.740858e-04,0.000077,0.000072
2001-08-01,-0.062066,3.852131e-03,0.000051,0.000054
2001-10-01,0.015148,2.294622e-04,0.000121,0.000187
2001-11-01,0.076536,5.857711e-03,0.000196,0.000154
2002-02-01,-0.019369,3.751727e-04,0.000031,0.000079
2002-03-01,0.028869,8.334460e-04,0.000092,0.000101


In [22]:
data['pred_ret_sqr'] = np.NaN
data['pred_var'] = np.NaN
for i in range(2, data.shape[0]):
    X = data['var_lag'].iloc[:i]
    Y = data['ret_sqr'].iloc[:i]
    regres = smf.OLS(Y, sm.add_constant(X)).fit()
    data['pred_ret_sqr'].iloc[i] = regres.params[0] + regres.params[1] * data['var_lag'].iloc[i]
    data['pred_var'].iloc[i] = data['pred_ret_sqr'].iloc[i]
    


/Users/apple/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [23]:
expect_ret = 0.1/250
A = 3
data['risky_weight'] = expect_ret/(A * data['pred_var'])

In [24]:
irx['date'] = irx.index
data = data.merge(irx, how = 'left', left_on = 'date', right_on = 'date')

In [25]:
data.dropna(inplace=True)

In [26]:
data = data.drop(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], axis = 1)

In [27]:
data = data.rename(index = str, columns = {'ret_x':'risky_ret', 'ret_y':'risk_free'})

In [28]:
data['portfolio_ret'] = data['risky_weight'] *(data['risky_ret'] + data['risk_free']) + (1 - data['risky_weight']) * data['risk_free']

In [29]:
data.head()

,risky_ret,ret_sqr,date,rv5,var_lag,pred_ret_sqr,pred_var,risky_weight,risk_free,portfolio_ret
2,-0.008555,0.000073,2001-05-01,0.000095,0.000079,0.011739,0.011739,0.011359,0.002950,0.002853
3,-0.029565,0.000874,2001-06-01,0.000077,0.000072,0.004161,0.004161,0.032043,0.002967,0.002019
4,-0.062066,0.003852,2001-08-01,0.000051,0.000054,0.002437,0.002437,0.054704,0.002733,-0.000662
5,0.015148,0.000229,2001-10-01,0.000121,0.000187,0.004845,0.004845,0.027519,0.001675,0.002092
6,0.076536,0.005858,2001-11-01,0.000196,0.000154,0.003134,0.003134,0.042547,0.001442,0.004698


In [30]:
data['port_exce_ret'] = data['portfolio_ret'] - data['risk_free']


In [31]:
print(data['port_exce_ret'].mean())

0.0013590537777614362


In [32]:
print(data['port_exce_ret'].std())

0.003722922825071678


In [33]:
sharp_ratio = data['port_exce_ret'].mean()/data['port_exce_ret'].std()
print(sharp_ratio)

0.36505021501090884


In [34]:
data['mov_avg_var'] = data['pred_var'].rolling(10).mean()

In [35]:
data['new_w'] = expect_ret/(A * data['mov_avg_var'])

In [36]:
data['new_port_ret'] = data['new_w'] * (data['risky_ret'] + data['risk_free']) + (1 - data['new_w']) * data['risk_free'] - data['risk_free']

In [37]:
print(data['new_port_ret'].mean())

0.0009384385564808471


In [38]:
print(data['new_port_ret'].std())

0.0031258538852890045
